In [1]:
## DO NOT ERASE THIS. IMPORTANT TO CORRECTLY IMPORT MODULES
import sys

sys.path.append("../")
sys.executable

'/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/bin/python3.12'

In [2]:
import pandas as pd

pd.set_option("display.max_rows", 102)
from tqdm.auto import tqdm

import torch
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import OrdinalEncoder, StandardScaler

from src.utils import (
    load_MovieLens,
    create_test_user,
    train_test_val_split,
    seed_everything,
    trainDatasetWithNumCatFeatures,
    split_test_df,
    add_not_watched_movies,
    create_test_user_display_df,
    train,
    predict,
)
from src.models import DCNv2
from src.metrics import reccomendation_report

## Constants

In [3]:
DATA_FOLDER = "../../data/ml-1m/"
RANDOM_STATE = 7

In [4]:
seed_everything(RANDOM_STATE)

## Data

In [5]:
df_users, df_movies, df_ratings = load_MovieLens(DATA_FOLDER)

### Creating a test_user

In [6]:
df_users, df_ratings, new_user_id = create_test_user(
    df_users, df_ratings, [6, 16, 3192, 1461, 827, 887, 593]
)
print("Test user watch list:")
display(df_ratings[df_ratings.userId == new_user_id].merge(df_movies, on="movieId"))

Test user watch list:


,userId,movieId,rating,timestamp,name,genre
0,6040,6,5,0,Sabrina (1995),Comedy|Romance
1,6040,16,5,1,Sense and Sensibility (1995),Drama|Romance
2,6040,3192,5,2,Singles (1992),Comedy|Drama|Romance
3,6040,1461,5,3,Love and Other Catastrophes (1996),Romance
4,6040,827,5,4,Emma (1996),Comedy|Drama|Romance
5,6040,887,5,5,Singin' in the Rain (1952),Musical|Romance
6,6040,593,5,6,Pretty Woman (1990),Comedy|Romance


### Train-test split
Methodology: Last user interaction is a test item. The rest is train. Validation part is 20% of test.

In [7]:
df_train, df_test, df_val = train_test_val_split(df_ratings, df_movies, RANDOM_STATE)

Enriching test:   0%|          | 0/4832 [00:00<?, ?it/s]

Enriching val:   0%|          | 0/1209 [00:00<?, ?it/s]

### Loading additional data

In [8]:
# cos_dist = computeCosineSimilarities(df_train, "userId", "movieId", df_movies["movieId"].nunique())

# with open("../../data/cos_dist.pt", "wb") as f:
#     torch.save(cos_dist, f)

with open("../../data/cos_dist.pt", "rb") as f:
    cos_dist = torch.load(f)
popularity = torch.bincount(
    torch.tensor(df_train.movieId.values), minlength=df_movies["movieId"].nunique()
)
popularity = popularity / torch.max(popularity)

### Building ordinal encoded features and normalizing continious features

In [9]:
ord_user = OrdinalEncoder()
user_cat = torch.tensor(ord_user.fit_transform(df_users[["gender", "occupation"]]))

ord_movie = OrdinalEncoder()
movie_cat = torch.tensor(ord_movie.fit_transform(df_movies[["genre"]]))

ss_user = StandardScaler()
user_num = torch.tensor(ss_user.fit_transform(df_users[["age"]]))

# features for val
user_cat_val = user_cat[df_val["userId"].values].clone().detach().to(torch.long)
user_num_val = user_num[df_val["userId"].values].clone().detach().to(torch.float)
movie_cat_val = movie_cat[df_val["movieId"].values].clone().detach().to(torch.long)
val_cat = torch.hstack((user_cat_val, movie_cat_val))

# features for test
user_cat_test = user_cat[df_test["userId"].values].clone().detach().to(torch.long)
user_num_test = user_num[df_test["userId"].values].clone().detach().to(torch.float)
movie_cat_test = movie_cat[df_test["movieId"].values].clone().detach().to(torch.long)
test_cat = torch.hstack((user_cat_test, movie_cat_test))


num_numeric_feats = 1
cat_feature_vocab = [len(i) for i in ord_user.categories_] + [
    len(i) for i in ord_movie.categories_
]

## DCN V2

In [10]:
seed_everything(RANDOM_STATE)
train_loader = DataLoader(
    trainDatasetWithNumCatFeatures(
        df_train, df_movies["movieId"].nunique(), user_cat, user_num, movie_cat
    ),
    batch_size=2048,
    shuffle=True,
)
val_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_val["userId"]),
        torch.tensor(df_val["movieId"]),
        user_num_val,
        val_cat,
    ),
    batch_size=4096,
    shuffle=False,
)
test_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_test["userId"]),
        torch.tensor(df_test["movieId"]),
        user_num_test,
        test_cat,
    ),
    batch_size=4096,
    shuffle=False,
)
num_users = df_users["userId"].nunique()
num_items = df_movies["movieId"].nunique()

  0%|          | 0/994175 [00:00<?, ?it/s]

In [11]:
seed_everything(RANDOM_STATE)
dcn = DCNv2(
    num_users,
    num_items,
    num_numeric_feats,
    cat_feature_vocab,
    l=4,
    n_mlp_layers=4,
    mlp_layers_dim=32,
    mlp_kwargs={
        "activation": True,
        "dropout": True,
        "batchnorm": True,
        "dropout_rate": 0.6,
    },
)
display(dcn)

DCNv2(
  (user_embedding): Embedding(6041, 5)
  (item_embedding): Embedding(3883, 5)
  (numerical_embeddings): ModuleList(
    (0): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=1, out_features=5, bias=True)
      )
    )
  )
  (categorical_embeddings): ModuleList(
    (0): Embedding(2, 5)
    (1): Embedding(21, 5)
    (2): Embedding(301, 5)
  )
  (mlp): Sequential(
    (MLP_layer_0): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=30, out_features=32, bias=True)
        (Activation): ReLU()
        (Dropout): Dropout(p=0.2, inplace=False)
        (BatchNorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (MLP_layer_1): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=32, out_features=32, bias=True)
        (Activation): ReLU()
        (Dropout): Dropout(p=0.6, inplace=False)
        (BatchNorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [12]:
optimizer = torch.optim.Adam(dcn.parameters(), lr=3e-3)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)
criterion = torch.nn.BCELoss()
n_epochs = 10
device = "cpu"

In [13]:
%%time
train(
    dcn,
    train_loader,
    optimizer,
    scheduler,
    criterion,
    n_epochs,
    val_loader=val_loader,
    df_val=df_val,
    cos_dist=cos_dist,
    popularity=popularity,
    verbose=True,
)

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 0
Train loss: 0.36806


Inference:   0%|          | 0/30 [00:00<?, ?batch/s]

{'Hit rate @ 15': tensor(0.5873),
 'NDCG @ 15': tensor(0.2811),
 'Diversity (ILD)': tensor(0.1406),
 'Novelty (EPC)': tensor(0.7008)}


...

{'Hit rate @ 15': tensor(0.6394),
 'NDCG @ 15': tensor(0.3198),
 'Diversity (ILD)': tensor(0.1302),
 'Novelty (EPC)': tensor(0.7299)}


  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 9
Train loss: 0.30701


Inference:   0%|          | 0/30 [00:00<?, ?batch/s]

{'Hit rate @ 15': tensor(0.6410),
 'NDCG @ 15': tensor(0.3221),
 'Diversity (ILD)': tensor(0.1303),
 'Novelty (EPC)': tensor(0.7302)}
CPU times: user 29min 48s, sys: 1h 23min 42s, total: 1h 53min 30s
Wall time: 13min 40s


In [14]:
df_test["pred"] = predict(dcn, test_loader, verbose=False).numpy()
pred, target, pred_items = split_test_df(df_test, "userId", "movieId", "pred", "action")
reccomendation_report(pred, target, pred_items, cos_dist, popularity, k=15)

{'Hit rate @ 15': tensor(0.6312),
 'NDCG @ 15': tensor(0.3186),
 'Diversity (ILD)': tensor(0.1311),
 'Novelty (EPC)': tensor(0.7275)}

## Test user recommendations

In [15]:
df_test_user = add_not_watched_movies(new_user_id, df_test, df_train, df_movies)

# features for test user
user_cat_test_user = (
    user_cat[df_test_user["userId"].values].clone().detach().to(torch.long)
)
user_num_test_user = (
    user_num[df_test_user["userId"].values].clone().detach().to(torch.float)
)
movie_cat_test_user = (
    movie_cat[df_test_user["movieId"].values].clone().detach().to(torch.long)
)
test_user_cat = torch.hstack((user_cat_test_user, movie_cat_test_user))

test_user_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_test_user["userId"]),
        torch.tensor(df_test_user["movieId"]),
        user_num_test_user,
        test_user_cat,
    ),
    batch_size=4096,
    shuffle=False,
)

In [16]:
df_test_user["pred"] = predict(dcn, test_user_loader, verbose=True).numpy()
display(create_test_user_display_df(df_test_user, df_movies, "pred"))

Inference:   0%|          | 0/1 [00:00<?, ?batch/s]

,userId,movieId,name,genre,pred
0,6040,2222,Edward Scissorhands (1990),Drama|Romance,0.944912
1,6040,1899,"Breakfast Club, The (1985)",Comedy|Drama,0.924641
2,6040,10,"American President, The (1995)",Comedy|Drama|Romance,0.881600
3,6040,1081,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi,0.864544
4,6040,1656,Good Will Hunting (1997),Drama,0.857766
5,6040,2252,Pleasantville (1998),Comedy,0.844122
6,6040,523,Schindler's List (1993),Drama|War,0.840052
7,6040,1854,There's Something About Mary (1998),Comedy,0.828717
8,6040,2728,Big (1988),Comedy|Fantasy,0.820365
9,6040,1720,"Wedding Singer, The (1998)",Comedy|Romance,0.789821
